In [1]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET

In [4]:
import numpy as np
dim = [52, 26, 13]
test = np.zeros((dim[l], 2) for l in range(3))

TypeError: expected sequence object with len >= 0 or a single integer

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [ ]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    member[0].text,
                    int(member[4][0].text),
                    int(member[4][1].text),
                    int(member[4][2].text),
                    int(member[4][3].text)
                    )
            xml_list.append(value)

    column_name = ['filename', 'width', 'height', 'cell_type', 'xmin', 'ymin', 'xmax', 'ymax']

    xml_df = pd.DataFrame(xml_list, columns=column_name)

    for i in range(len(xml_df['filename'])):
        if '.jpg' not in xml_df.filename[i]:
            xml_df.filename[i] = xml_df.filename[i] + '.jpg'
    return xml_df

In [ ]:
path = './BCCD-RBC-WBC-differentiation/Annotations/'

In [ ]:
df = xml_to_csv(path)

In [ ]:
df.head()

In [ ]:
new_df = pd.DataFrame()
new_df['class'] = df.cell_type
new_df = pd.get_dummies(new_df)
new_df = new_df.rename(columns={"class_Platelets": "Platelet", 
                       "class_RBC": "RBC", 
                       "class_WBC": "WBC"})
new_df.insert(0, 'confidence', 1)

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [3]:
path = './BCCD-RBC-WBC-differentiation/Annotations/'

In [4]:
df = xml_to_csv(path)

In [5]:
df.head()

,filename,width,height,cell_type,xmin,ymin,xmax,ymax
0,BloodImage_00000.jpg,640,480,WBC,260,177,491,376
1,BloodImage_00000.jpg,640,480,RBC,78,336,184,435
2,BloodImage_00000.jpg,640,480,RBC,63,237,169,336
3,BloodImage_00000.jpg,640,480,RBC,214,362,320,461
4,BloodImage_00000.jpg,640,480,RBC,414,352,506,445


In [ ]:
new_df = pd.DataFrame()
new_df['class'] = df.cell_type
new_df = pd.get_dummies(new_df)
new_df = new_df.rename(columns={"class_Platelets": "Platelet", 
                       "class_RBC": "RBC", 
                       "class_WBC": "WBC"})
new_df.insert(0, 'confidence', 1)

In [12]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
import cv2
import matplotlib.pyplot as plt

In [ ]:
def crop_to_WBC_Img(df, path, img_name):
    img = cv2.imread(path + img_name)
    plt.imshow(img)
    img_data = df[(df['filename'] == img_name) & (df['cell_type'] == 'WBC')]
    xmin = img_data['xmin'][0]
    xmax = img_data['xmax'][0]
    ymin = img_data['ymin'][0]
    ymax = img_data['ymax'][0]
    cropped_img = img[ymin:ymax, xmin:xmax]
    plt.imshow(cropped_img)
    return

crop_to_WBC_Img(df, './BCCD-RBC-WBC-differentiation/JPEGImages/', 'BloodImage_00000.jpg')